# Build a Contextual Retrieval based RAG System

## Install OpenAI, and LangChain dependencies

In [1]:
!pip install langchain==0.3.10
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11
!pip install jq==1.8.0
!pip install pymupdf==1.25.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.2/390.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.7 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.22
    Uninstalling langchain-core-0.3.22:
      Successfully uninstalled langchain-core-0.3.22
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.10
    Uninstalling langchain

## Install Chroma Vector DB and LangChain wrapper

In [2]:
!pip install langchain-chroma==0.1.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 85.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 6.3 MB/s eta 0:00:

## Enter Open AI API Key

In [3]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

Enter Open AI API Key: ··········


## Setup Environment Variables

In [4]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

### Open AI Embedding Models

LangChain enables us to access Open AI embedding models which include the newest models: a smaller and highly efficient `text-embedding-3-small` model, and a larger and more powerful `text-embedding-3-large` model.

In [5]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

## Loading and Processing the Data

### Get the dataset

In [ ]:
# if you can't download using the following code
# go to https://drive.google.com/file/d/1aZxZejfteVuofISodUrY2CDoyuPLYDGZ download it
# manually upload it on colab
!gdown 1aZxZejfteVuofISodUrY2CDoyuPLYDGZ

Downloading...
From: https://drive.google.com/uc?id=1aZxZejfteVuofISodUrY2CDoyuPLYDGZ
To: /content/rag_docs.zip
100% 5.92M/5.92M [00:00<00:00, 35.1MB/s]


In [ ]:
!unzip rag_docs.zip

Archive:  rag_docs.zip
   creating: rag_docs/
  inflating: rag_docs/attention_paper.pdf  
  inflating: rag_docs/cnn_paper.pdf  
  inflating: rag_docs/resnet_paper.pdf  
  inflating: rag_docs/vision_transformer.pdf  
  inflating: rag_docs/wikidata_rag_demo.jsonl  


### Load and Process JSON Documents

In [ ]:
from langchain.document_loaders import JSONLoader

loader = JSONLoader(file_path='./rag_docs/wikidata_rag_demo.jsonl',
                    jq_schema='.',
                    text_content=False,
                    json_lines=True)
wiki_docs = loader.load()

In [ ]:
len(wiki_docs)

1801

In [ ]:
wiki_docs[3]

Document(metadata={'source': '/content/rag_docs/wikidata_rag_demo.jsonl', 'seq_num': 4}, page_content='{"id": "71548", "title": "Chi-square distribution", "paragraphs": ["In probability theory and statistics, the chi-square distribution (also chi-squared or formula_1\\u00a0 distribution) is one of the most widely used theoretical probability distributions. Chi-square distribution with formula_2 degrees of freedom is written as formula_3. It is a special case of gamma distribution.", "Chi-square distribution is primarily used in statistical significance tests and confidence intervals. It is useful, because it is relatively easy to show that certain probability distributions come close to it, under certain conditions. One of these conditions is that the null hypothesis must be true. Another one is that the different random variables (or observations) must be independent of each other."]}')

In [ ]:
import json
from langchain.docstore.document import Document
wiki_docs_processed = []

for doc in wiki_docs:
    doc = json.loads(doc.page_content)
    metadata = {
        "title": doc['title'],
        "id": doc['id'],
        "source": "Wikipedia",
        "page": 1
    }
    data = ' '.join(doc['paragraphs'])
    wiki_docs_processed.append(Document(page_content=data, metadata=metadata))

In [ ]:
wiki_docs_processed[3]

Document(metadata={'title': 'Chi-square distribution', 'id': '71548', 'source': 'Wikipedia', 'page': 1}, page_content='In probability theory and statistics, the chi-square distribution (also chi-squared or formula_1\xa0 distribution) is one of the most widely used theoretical probability distributions. Chi-square distribution with formula_2 degrees of freedom is written as formula_3. It is a special case of gamma distribution. Chi-square distribution is primarily used in statistical significance tests and confidence intervals. It is useful, because it is relatively easy to show that certain probability distributions come close to it, under certain conditions. One of these conditions is that the null hypothesis must be true. Another one is that the different random variables (or observations) must be independent of each other.')

### Load and Process PDF documents

#### Create Chunk Contexts for Contextual Retrieval

![](https://i.imgur.com/LRhKHzk.png)

In [ ]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [ ]:
# create chunk context generation chain
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser


def generate_chunk_context(document, chunk):

    chunk_process_prompt = """You are an AI assistant specializing in research paper analysis.
                            Your task is to provide brief, relevant context for a chunk of text
                            based on the following research paper.

                            Here is the research paper:
                            <paper>
                            {paper}
                            </paper>

                            Here is the chunk we want to situate within the whole document:
                            <chunk>
                            {chunk}
                            </chunk>

                            Provide a concise context (3-4 sentences max) for this chunk,
                            considering the following guidelines:

                            - Give a short succinct context to situate this chunk within the overall document
                            for the purposes of improving search retrieval of the chunk.
                            - Answer only with the succinct context and nothing else.
                            - Context should be mentioned like 'Focuses on ....'
                            do not mention 'this chunk or section focuses on...'

                            Context:
                        """

    prompt_template = ChatPromptTemplate.from_template(chunk_process_prompt)

    agentic_chunk_chain = (prompt_template
                                |
                            chatgpt
                                |
                            StrOutputParser())

    context = agentic_chunk_chain.invoke({'paper': document, 'chunk': chunk})

    return context

In [ ]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import uuid

def create_contextual_chunks(file_path, chunk_size=3500, chunk_overlap=0):

    print('Loading pages:', file_path)
    loader = PyMuPDFLoader(file_path)
    doc_pages = loader.load()

    print('Chunking pages:', file_path)
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size,
                                              chunk_overlap=chunk_overlap)
    doc_chunks = splitter.split_documents(doc_pages)

    print('Generating contextual chunks:', file_path)
    original_doc = '\n'.join([doc.page_content for doc in doc_chunks])
    contextual_chunks = []
    for chunk in doc_chunks:
        chunk_content = chunk.page_content
        chunk_metadata = chunk.metadata
        chunk_metadata_upd = {
            'id': str(uuid.uuid4()),
            'page': chunk_metadata['page'],
            'source': chunk_metadata['source'],
            'title': chunk_metadata['source'].split('/')[-1]
        }
        context = generate_chunk_context(original_doc, chunk_content)
        contextual_chunks.append(Document(page_content=context+'\n'+chunk_content,
                                          metadata=chunk_metadata_upd))
    print('Finished processing:', file_path)
    print()
    return contextual_chunks

In [ ]:
from glob import glob

pdf_files = glob('./rag_docs/*.pdf')
pdf_files

['./rag_docs/resnet_paper.pdf',
 './rag_docs/vision_transformer.pdf',
 './rag_docs/cnn_paper.pdf',
 './rag_docs/attention_paper.pdf']

In [ ]:
paper_docs = []
for fp in pdf_files:
    paper_docs.extend(create_contextual_chunks(file_path=fp, chunk_size=3500))

Loading pages: ./rag_docs/resnet_paper.pdf
Chunking pages: ./rag_docs/resnet_paper.pdf
Generating contextual chunks: ./rag_docs/resnet_paper.pdf
Finished processing: ./rag_docs/resnet_paper.pdf

Loading pages: ./rag_docs/vision_transformer.pdf
Chunking pages: ./rag_docs/vision_transformer.pdf
Generating contextual chunks: ./rag_docs/vision_transformer.pdf
Finished processing: ./rag_docs/vision_transformer.pdf

Loading pages: ./rag_docs/cnn_paper.pdf
Chunking pages: ./rag_docs/cnn_paper.pdf
Generating contextual chunks: ./rag_docs/cnn_paper.pdf
Finished processing: ./rag_docs/cnn_paper.pdf

Loading pages: ./rag_docs/attention_paper.pdf
Chunking pages: ./rag_docs/attention_paper.pdf
Generating contextual chunks: ./rag_docs/attention_paper.pdf
Finished processing: ./rag_docs/attention_paper.pdf



In [ ]:
len(paper_docs)

79

In [ ]:
paper_docs[0]

Document(metadata={'id': 'ac10b59a-79a9-4e24-9ae7-34ff0403dc3a', 'page': 0, 'source': './rag_docs/resnet_paper.pdf', 'title': 'resnet_paper.pdf'}, page_content='Focuses on the introduction of deep residual learning as a framework to facilitate the training of significantly deeper neural networks, addressing challenges such as vanishing gradients and degradation of accuracy. It highlights the empirical success of residual networks on the ImageNet dataset and their application in various visual recognition tasks, including object detection and segmentation, leading to top performances in major competitions.\nDeep Residual Learning for Image Recognition\nKaiming He\nXiangyu Zhang\nShaoqing Ren\nJian Sun\nMicrosoft Research\n{kahe, v-xiangz, v-shren, jiansun}@microsoft.com\nAbstract\nDeeper neural networks are more difﬁcult to train. We\npresent a residual learning framework to ease the training\nof networks that are substantially deeper than those used\npreviously. We explicitly reformula

### Combine all document chunks in one list

In [ ]:
len(wiki_docs_processed)

1801

In [ ]:
total_docs = wiki_docs_processed + paper_docs
len(total_docs)

1880

## Index Document Chunks and Embeddings in Vector DB

Here we initialize a connection to a Chroma vector DB client, and also we want to save to disk, so we simply initialize the Chroma client and pass the directory where we want the data to be saved to.

In [ ]:
from langchain_chroma import Chroma

# create vector DB of docs and embeddings - takes < 30s on Colab
chroma_db = Chroma.from_documents(documents=total_docs,
                                  collection_name='my_context_db',
                                  embedding=openai_embed_model,
                                  # need to set the distance function to cosine else it uses euclidean by default
                                  # check https://docs.trychroma.com/guides#changing-the-distance-function
                                  collection_metadata={"hnsw:space": "cosine"},
                                  persist_directory="./my_context_db")

### Load Vector DB from disk

This is just to show once you have a vector database on disk you can just load and create a connection to it anytime

In [ ]:
# load from disk
chroma_db = Chroma(persist_directory="./my_context_db",
                   collection_name='my_context_db',
                   embedding_function=openai_embed_model)

In [ ]:
chroma_db

### Semantic Similarity based Retrieval

We use simple cosine similarity here and retrieve the top 5 similar documents based on the user input query

In [ ]:
similarity_retriever = chroma_db.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 5})

In [ ]:
from IPython.display import display, Markdown

def display_docs(docs):
    for doc in docs:
        print('Metadata:', doc.metadata)
        print('Content Brief:')
        display(Markdown(doc.page_content[:1000]))
        print()

In [ ]:
query = "what is machine learning?"
top_docs = similarity_retriever.invoke(query)
display_docs(top_docs)

Metadata: {'id': '564928', 'page': 1, 'source': 'Wikipedia', 'title': 'Machine learning'}
Content Brief:


Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). It is a subfield of computer science. The idea came from work in artificial intelligence. Machine learning explores the study and construction of algorithms which can learn and make predictions on data. Such algorithms follow programmed instructions, but can also make predictions or decisions based on data. They build a model from sample inputs. Machine learning is done where designing and programming explicit algorithms cannot be done. Examples include spam filtering, detection of network intruders or malicious insiders working towards a data breach, optical character recognition (OCR), search engines and computer vision.


Metadata: {'id': '359370', 'page': 1, 'source': 'Wikipedia', 'title': 'Supervised learning'}
Content Brief:


In machine learning, supervised learning is the task of inferring a function from labelled training data. The results of the training are known beforehand, the system simply learns how to get to these results correctly. Usually, such systems work with vectors. They get the training data and the result of the training as two vectors and produce a "classifier". Usually, the system uses inductive reasoning to generalize the training data.


Metadata: {'id': '663523', 'page': 1, 'source': 'Wikipedia', 'title': 'Deep learning'}
Content Brief:


Deep learning (also called deep structured learning or hierarchical learning) is a kind of machine learning, which is mostly used with certain kinds of neural networks. As with other kinds of machine-learning, learning sessions can be unsupervised, semi-supervised, or supervised. In many cases, structures are organised so that there is at least one intermediate layer (or hidden layer), between the input layer and the output layer. Certain tasks, such as as recognizing and understanding speech, images or handwriting, is easy to do for humans. However, for a computer, these tasks are very difficult to do. In a multi-layer neural network (having more than two layers), the information processed will become more abstract with each added layer. Deep learning models are inspired by information processing and communication patterns in biological nervous systems; they are different from the structural and functional properties of biological brains (especially the human brain) in many ways, whic


Metadata: {'id': '6360', 'page': 1, 'source': 'Wikipedia', 'title': 'Artificial intelligence'}
Content Brief:


Artificial intelligence (AI) is the ability of a computer program or a machine to think and learn. It is also a field of study which tries to make computers "smart". They work on their own without being encoded with commands. John McCarthy came up with the name "Artificial Intelligence" in 1955. In general use, the term "artificial intelligence" means a programme which mimics human cognition. At least some of the things we associate with other minds, such as learning and problem solving can be done by computers, though not in the same way as we do. Andreas Kaplan and Michael Haenlein define AI as a system’s ability to correctly interpret external data, to learn from such data, and to use those learnings to achieve specific goals and tasks through flexible adaptation. An ideal (perfect) intelligent machine is a flexible agent which perceives its environment and takes actions to maximize its chance of success at some goal or objective. As machines become increasingly capable, mental facu


Metadata: {'id': '44742', 'page': 1, 'source': 'Wikipedia', 'title': 'Artificial neural network'}
Content Brief:


A neural network (also called an ANN or an artificial neural network) is a sort of computer software, inspired by biological neurons. Biological brains are capable of solving difficult problems, but each neuron is only responsible for solving a very small part of the problem. Similarly, a neural network is made up of cells that work together to produce a desired result, although each individual cell is only responsible for solving a small part of the problem. This is one method for creating artificially intelligent programs. Neural networks are an example of machine learning, where a program can change as it learns to solve a problem. A neural network can be trained and improved with each example, but the larger the neural network, the more examples it needs to perform well—often needing millions or billions of examples in the case of deep learning. There are two ways to think of a neural network. First is like a human brain. Second is like a mathematical equation.

In [ ]:
query = "what is the difference between transformers and vision transformers?"
top_docs = similarity_retriever.invoke(query)
display_docs(top_docs)

Metadata: {'id': '0a4aa65e-ba24-40c0-890f-7143ff77e6f4', 'page': 0, 'source': './rag_docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf'}
Content Brief:


Focuses on the introduction of the Vision Transformer (ViT) model, which applies a pure Transformer architecture to image classification tasks by treating image patches as tokens. It highlights the limitations of traditional convolutional networks in computer vision and presents the advantages of using Transformers, particularly when pre-trained on large datasets. The section sets the stage for the subsequent exploration of ViT's performance compared to state-of-the-art convolutional networks.
Published as a conference paper at ICLR 2021
AN IMAGE IS WORTH 16X16 WORDS:
TRANSFORMERS FOR IMAGE RECOGNITION AT SCALE
Alexey Dosovitskiy∗,†, Lucas Beyer∗, Alexander Kolesnikov∗, Dirk Weissenborn∗,
Xiaohua Zhai∗, Thomas Unterthiner, Mostafa Dehghani, Matthias Minderer,
Georg Heigold, Sylvain Gelly, Jakob Uszkoreit, Neil Houlsby∗,†
∗equal technical contribution, †equal advising
Google Research, Brain Team
{adosovitskiy, neilhoulsby}@google.com
ABSTRACT
While the Transformer architecture has becom


Metadata: {'id': '38169299-0ef0-4af7-a2cb-506caedabf6d', 'page': 7, 'source': './rag_docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf'}
Content Brief:


Focuses on a controlled scaling study of various models, including Vision Transformers and ResNets, evaluating their transfer performance from the JFT-300M dataset. It highlights the performance versus pre-training cost, revealing that Vision Transformers generally outperform ResNets in terms of efficiency and scalability, while also discussing the implications for future model scaling efforts.
Published as a conference paper at ICLR 2021
4.4
SCALING STUDY
We perform a controlled scaling study of different models by evaluating transfer performance from
JFT-300M. In this setting data size does not bottleneck the models’ performances, and we assess
performance versus pre-training cost of each model. The model set includes: 7 ResNets, R50x1,
R50x2 R101x1, R152x1, R152x2, pre-trained for 7 epochs, plus R152x2 and R200x3 pre-trained
for 14 epochs; 6 Vision Transformers, ViT-B/32, B/16, L/32, L/16, pre-trained for 7 epochs, plus
L/16 and H/14 pre-trained for 14 epochs; and 5 hybrids, R50+ViT


Metadata: {'id': '9efab417-6594-4327-af0e-a8229e183a97', 'page': 2, 'source': './rag_docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf'}
Content Brief:


Focuses on the architecture and methodology of the Vision Transformer (ViT), detailing how images are processed by splitting them into patches, embedding them, and utilizing a standard Transformer encoder for image classification tasks. It describes the model's design principles, including the use of position embeddings and the integration of a classification token, while referencing foundational work in Transformer architecture.
Published as a conference paper at ICLR 2021
Transformer Encoder
MLP 
Head
Vision Transformer (ViT)
*
Linear Projection of Flattened Patches
* Extra learnable
     [ cl ass]  embedding
1
2
3
4
5
6
7
8
9
0
Patch + Position 
Embedding
Class
Bird
Ball
Car
...
Embedded 
Patches
Multi-Head 
Attention
Norm
MLP
Norm
+
L x
+
Transformer Encoder
Figure 1: Model overview. We split an image into ﬁxed-size patches, linearly embed each of them,
add position embeddings, and feed the resulting sequence of vectors to a standard Transformer
encoder. In order to perform classiﬁ


Metadata: {'id': 'c23af995-486e-4fc9-8762-5fb229441a61', 'page': 7, 'source': './rag_docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf'}
Content Brief:


Focuses on the behavior of attention mechanisms in the Vision Transformer (ViT), highlighting how attention distances vary across layers and the implications for feature extraction. It compares the attention patterns in ViT to those in hybrid models that incorporate convolutional networks, suggesting that early layers in CNNs may perform a similar role to localized attention in ViT. Additionally, it notes the relevance of attention distance to the model's ability to focus on semantically important regions for classification tasks.
have consistently small attention distances in the low layers. This highly localized attention is
less pronounced in hybrid models that apply a ResNet before the Transformer (Figure 7, right),
suggesting that it may serve a similar function as early convolutional layers in CNNs. Further, the
attention distance increases with network depth. Globally, we ﬁnd that the model attends to image
regions that are semantically relevant for classiﬁcation (Figure 6).
4.6


Metadata: {'id': '9e785ba7-2575-4e74-a3d6-6edefb9053e8', 'page': 1, 'source': './rag_docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf'}
Content Brief:


Focuses on the performance of the Vision Transformer (ViT) in image recognition tasks, highlighting its ability to achieve state-of-the-art results when pre-trained on large datasets like ImageNet-21k and JFT-300M. It contrasts the inductive biases of convolutional neural networks (CNNs) with the advantages of large-scale training for ViT, demonstrating its competitive accuracy across various benchmarks.
Published as a conference paper at ICLR 2021
inherent to CNNs, such as translation equivariance and locality, and therefore do not generalize well
when trained on insufﬁcient amounts of data.
However, the picture changes if the models are trained on larger datasets (14M-300M images). We
ﬁnd that large scale training trumps inductive bias. Our Vision Transformer (ViT) attains excellent
results when pre-trained at sufﬁcient scale and transferred to tasks with fewer datapoints. When
pre-trained on the public ImageNet-21k dataset or the in-house JFT-300M dataset, ViT approaches
or beats st

## Build the RAG Pipeline

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = """You are an assistant who is an expert in question-answering tasks.
                Answer the following question using only the following pieces of retrieved context.
                If the answer is not in the context, do not make up answers, just say that you don't know.
                Keep the answer detailed and well formatted based on the information from the context.

                Question:
                {question}

                Context:
                {context}

                Answer:
            """

rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_rag_chain = (
    {
        "context": (similarity_retriever
                      |
                    format_docs),
        "question": RunnablePassthrough()
    }
      |
    rag_prompt_template
      |
    chatgpt
)

In [ ]:
from IPython.display import display, Markdown

query = "What is machine learning?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

Machine learning is a subfield of computer science that provides computers with the ability to learn without being explicitly programmed. The concept originated from work in artificial intelligence and involves the study and construction of algorithms that can learn from and make predictions based on data. These algorithms follow programmed instructions but can also make decisions or predictions based on the data they process. 

In machine learning, a model is built from sample inputs, and it is particularly useful in scenarios where designing and programming explicit algorithms is not feasible. Common applications of machine learning include spam filtering, detection of network intruders, optical character recognition (OCR), search engines, and computer vision.

There are different types of machine learning, including supervised learning, where the system infers a function from labeled training data. In this case, the results of the training are known beforehand, and the system learns to achieve these results correctly, often using inductive reasoning to generalize from the training data.

Additionally, deep learning is a specific kind of machine learning that primarily utilizes neural networks. It involves multiple layers of processing, allowing the model to learn increasingly abstract representations of the data. Deep learning is particularly effective for complex tasks such as speech recognition, image understanding, and handwriting recognition.

Overall, machine learning enables computers to adapt and improve their performance on tasks through experience, making it a crucial component of modern artificial intelligence systems.

In [ ]:
query = "What is a CNN?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

A Convolutional Neural Network (CNN) is an advanced architecture of Artificial Neural Networks (ANNs) specifically designed for image-driven pattern recognition tasks. CNNs are particularly effective in solving complex image analysis problems due to their unique structure and functionality.

### Key Features of CNNs:

1. **Three-Dimensional Neuron Organization**:
   - Unlike traditional ANNs, which typically have a two-dimensional structure, CNNs organize neurons in three dimensions: height, width, and depth. The depth dimension represents the activation volume rather than the total number of layers.

2. **Layer Types**:
   - CNNs are composed of three main types of layers:
     - **Convolutional Layers**: These layers apply a convolution operation to the input, allowing the network to learn spatial hierarchies of features. Each neuron in a convolutional layer connects to a small region of the input, which helps in detecting local patterns.
     - **Pooling Layers**: These layers reduce the spatial dimensionality of the input, helping to decrease the number of parameters and computations in the network, while also controlling overfitting.
     - **Fully-Connected Layers**: These layers connect every neuron in one layer to every neuron in the next layer, typically used at the end of the network to produce the final output.

3. **Architecture Design**:
   - A common practice in CNN architecture is to stack multiple convolutional layers before pooling layers. This stacking allows for the extraction of more complex features from the input data. Smaller convolutional layers are often preferred to manage computational complexity and memory usage.

4. **Input Handling**:
   - CNNs are designed to process image data, which requires careful consideration of input dimensionality. Techniques such as zero-padding are used to maintain the spatial dimensions of the input during convolution operations.

5. **Learning Paradigms**:
   - CNNs typically utilize supervised learning, where the model is trained on labeled data to minimize classification error. This is crucial for tasks like image classification, where the goal is to accurately identify objects within images.

### Applications:
CNNs are widely used in various applications, including but not limited to:
- Image classification
- Object detection
- Image segmentation
- Facial recognition

In summary, CNNs represent a significant advancement in the field of machine learning, particularly for tasks involving image data, by leveraging their specialized architecture to efficiently learn and recognize patterns within images.

In [ ]:
query = "How is a resnet better than a CNN?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

A ResNet (Residual Network) is considered better than a traditional CNN (Convolutional Neural Network) for several reasons, primarily due to its architectural innovations that address specific challenges faced by deeper networks. Here are the key advantages of ResNets over standard CNNs:

1. **Residual Learning Framework**: 
   - ResNets introduce the concept of residual learning, where the network learns to fit a residual mapping instead of the original unreferenced mapping. This is mathematically expressed as \( F(x) + x \), where \( F(x) \) is the residual function. This approach simplifies the optimization process, making it easier for the network to learn identity mappings when necessary.

2. **Shortcut Connections**:
   - The architecture of ResNets includes shortcut connections that skip one or more layers. These connections allow gradients to flow more easily during backpropagation, mitigating issues like vanishing gradients that can occur in very deep networks. This results in more effective training of deeper models.

3. **Degradation Problem**:
   - Traditional deep networks often suffer from the degradation problem, where adding more layers leads to higher training error. In contrast, ResNets can maintain or even improve performance as depth increases. For instance, the 34-layer ResNet outperformed the 34-layer plain network by achieving lower training error and better generalization.

4. **Higher Accuracy with Increased Depth**:
   - ResNets can achieve significantly better accuracy with increased depth without the degradation problem. For example, the 152-layer ResNet has been shown to outperform shallower networks and even other architectures like VGG, achieving a top-5 validation error of 4.49% in the ILSVRC 2015 competition.

5. **Lower Computational Complexity**:
   - Despite being deeper, ResNets can have lower computational complexity (measured in FLOPs) compared to other architectures like VGG. For instance, a 152-layer ResNet has 11.3 billion FLOPs, which is lower than the 15.3 billion FLOPs of VGG-16, while still achieving superior accuracy.

6. **Generalization Performance**:
   - ResNets have demonstrated excellent generalization capabilities across various recognition tasks, not just in image classification but also in object detection and segmentation tasks, as evidenced by their success in competitions like ILSVRC and COCO.

In summary, the architectural innovations of ResNets, particularly the use of residual learning and shortcut connections, enable them to train deeper networks more effectively, avoid degradation issues, and achieve higher accuracy compared to traditional CNNs.

In [ ]:
query = "What is NLP and its relation to linguistics?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

Natural Language Processing (NLP) is a field within Artificial Intelligence that focuses on enabling computers to automatically understand and generate human languages. The term "Natural Language" specifically refers to human languages, distinguishing them from programming languages. The overarching goal of NLP is to facilitate seamless interaction between humans and computers through language.

NLP is closely related to linguistics, which is the scientific study of language and its structure. Linguistics provides the foundational theories and frameworks that inform the development of NLP technologies. By leveraging insights from linguistics, NLP aims to decode the complexities of human language, including syntax, semantics, and pragmatics, to improve the accuracy and effectiveness of language processing tasks.

In summary, NLP is a crucial intersection of Artificial Intelligence and linguistics, aiming to enhance computer understanding and generation of human language.

In [ ]:
query = "What is the difference between AI, ML and DL?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

The difference between AI, ML, and DL can be summarized as follows:

### Artificial Intelligence (AI)
- **Definition**: AI refers to the ability of a computer program or machine to think and learn, mimicking human cognition. It encompasses a broad range of technologies and applications that enable machines to perform tasks that typically require human intelligence.
- **Scope**: AI is a field of study aimed at creating systems that can interpret external data, learn from it, and adapt to achieve specific goals. It includes various subfields, one of which is machine learning.
- **Examples**: AI applications can range from simple rule-based systems to complex algorithms that can learn and adapt over time.

### Machine Learning (ML)
- **Definition**: ML is a subfield of AI that focuses on the study and construction of algorithms that allow computers to learn from and make predictions based on data without being explicitly programmed.
- **Functionality**: ML algorithms build models from sample inputs and can make decisions or predictions based on new data. It is particularly useful in scenarios where traditional programming is impractical.
- **Examples**: Applications of ML include spam filtering, network intrusion detection, optical character recognition (OCR), and computer vision.

### Deep Learning (DL)
- **Definition**: DL is a specialized subset of machine learning that primarily uses neural networks with multiple layers (deep neural networks) to process data.
- **Characteristics**: In deep learning, the information processed becomes increasingly abstract with each added layer, allowing for complex tasks such as speech and image recognition. It is inspired by the structure and function of biological neural networks.
- **Examples**: DL is often used in applications that require high levels of abstraction, such as image and speech recognition, where traditional ML methods may struggle.

In summary, AI is the overarching field that includes both machine learning and deep learning, with machine learning being a method of achieving AI and deep learning being a more advanced technique within machine learning.

In [ ]:
query = "What is the difference between transformers and vision transformers?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

The difference between transformers and vision transformers (ViTs) primarily lies in their application and the way they process input data.

1. **Architecture and Input Handling**:
   - **Transformers**: Originally designed for natural language processing (NLP), transformers operate on sequences of tokens (words) and utilize self-attention mechanisms to capture relationships between these tokens. The input is typically a 1D sequence of embeddings.
   - **Vision Transformers (ViTs)**: ViTs adapt the transformer architecture for image classification tasks by treating image patches as tokens. An image is divided into fixed-size patches, which are then flattened and linearly embedded into a sequence. This sequence of patch embeddings is fed into a standard transformer encoder, allowing the model to process 2D image data effectively.

2. **Inductive Biases**:
   - **Transformers**: In NLP, transformers benefit from the sequential nature of text, where the order of tokens is crucial, and they leverage the inherent structure of language.
   - **Vision Transformers**: ViTs lack some of the inductive biases present in convolutional neural networks (CNNs), such as translation equivariance and locality. This means that while CNNs are designed to recognize patterns in spatial hierarchies, ViTs rely on large-scale training to learn these patterns from the data itself.

3. **Performance and Training**:
   - **Transformers**: In NLP, transformers have become the standard due to their ability to scale and perform well on large datasets.
   - **Vision Transformers**: ViTs have shown that when pre-trained on large datasets, they can achieve competitive or superior performance compared to state-of-the-art CNNs, particularly in image classification tasks. They require fewer computational resources to train while still achieving high accuracy on various benchmarks.

4. **Attention Mechanism**:
   - **Transformers**: The attention mechanism in transformers allows for global context understanding across the entire sequence of tokens.
   - **Vision Transformers**: ViTs utilize self-attention to integrate information across the entire image, enabling the model to focus on semantically important regions for classification. This capability is particularly beneficial in understanding complex visual patterns.

In summary, while both transformers and vision transformers share a foundational architecture, their differences lie in their input types, the biases they leverage, their performance characteristics, and how they apply attention mechanisms to their respective tasks.

In [ ]:
query = "How is self-attention important in transformers?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

Self-attention is crucial in transformers for several reasons, particularly in the context of the Vision Transformer (ViT) and its application in feature extraction and classification tasks. Here are the key points regarding the importance of self-attention in transformers:

1. **Attention Mechanism**: Self-attention allows the model to weigh the importance of different parts of the input data when making predictions. This is particularly beneficial in understanding contextual relationships within the data, whether it be in natural language processing or computer vision.

2. **Localized Attention**: In the Vision Transformer, early layers exhibit highly localized attention, which is essential for focusing on specific regions of an image that are semantically relevant for classification tasks. This behavior is somewhat analogous to the function of early convolutional layers in convolutional neural networks (CNNs), which also focus on localized features.

3. **Attention Distance**: The attention distance, which refers to how far apart the attended elements are in the input, varies across layers in the transformer. In lower layers, attention distances are consistently small, indicating a focus on local features. As the network depth increases, the attention distance grows, allowing the model to capture more global relationships and dependencies within the data.

4. **Feature Extraction**: The ability of self-attention to adaptively focus on semantically important regions enhances the model's feature extraction capabilities. This is critical for tasks such as image classification, where understanding the context and relationships between different parts of an image can significantly impact performance.

5. **Comparison with Hybrid Models**: The attention patterns in ViT are compared to those in hybrid models that incorporate CNNs. The findings suggest that while CNNs may perform localized attention through their early layers, transformers leverage self-attention to dynamically adjust their focus based on the input data, leading to potentially superior performance in certain tasks.

In summary, self-attention in transformers is vital for enabling the model to effectively capture both local and global dependencies in the data, enhancing its ability to perform complex tasks such as classification and feature extraction.

In [ ]:
query = "How does a resnet work?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

A ResNet, or Residual Network, operates on the principle of residual learning to address the degradation problem that occurs in deep neural networks. Here’s a detailed explanation of how it works:

### Key Concepts of ResNet

1. **Residual Mapping**:
   - Instead of learning the desired underlying mapping \( H(x) \) directly, ResNets learn a residual mapping \( F(x) = H(x) - x \). This means that the network is designed to learn the difference between the desired output and the input, which is often easier than learning the output directly.

2. **Shortcut Connections**:
   - ResNets utilize shortcut connections that skip one or more layers. These connections perform identity mapping, meaning they pass the input \( x \) directly to the output of the stacked layers. The output of the residual block is then given by \( F(x) + x \).
   - This approach allows gradients to flow through the network more effectively during backpropagation, mitigating issues like vanishing gradients that can occur in very deep networks.

3. **Identity Mapping**:
   - When the input and output dimensions are the same, the shortcut connection simply adds the input to the output of the stacked layers. If the dimensions differ, the shortcut can either pad the input with zeros or use a projection shortcut (e.g., 1x1 convolutions) to match the dimensions.

### Advantages of ResNet

- **Easier Optimization**: By reformulating the learning task to focus on residuals, ResNets make it easier to optimize deeper networks. This is evidenced by the fact that deeper ResNets (e.g., 34-layer, 110-layer) can achieve lower training errors compared to their plain counterparts of the same depth.
  
- **Higher Accuracy with Depth**: ResNets can effectively utilize increased depth to improve accuracy. For instance, a 34-layer ResNet outperforms an 18-layer ResNet, demonstrating that deeper networks can generalize better and achieve higher accuracy.

- **Lower Complexity**: Despite having more layers, ResNets can maintain lower computational complexity compared to traditional architectures like VGG, allowing for efficient training and inference.

### Empirical Evidence

- Experiments on datasets like ImageNet and CIFAR-10 show that ResNets not only overcome the degradation problem but also achieve significant accuracy improvements as depth increases. For example, a 152-layer ResNet has been shown to outperform previous models while having lower complexity.

In summary, ResNets leverage residual learning and shortcut connections to facilitate the training of very deep networks, leading to improved performance and easier optimization compared to traditional deep learning architectures.

In [ ]:
query = "What is LangGraph?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

I don't know.

In [ ]:
query = "What is an Agentic AI System?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

The context provided does not contain specific information about an "Agentic AI System." Therefore, I don't know what an Agentic AI System is.

In [ ]:
query = "What is LangChain?"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))

I don't know.

# Build a RAG System with Sources

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = """You are an assistant who is an expert in question-answering tasks.
                Answer the following question using only the following pieces of retrieved context.
                If the answer is not in the context, do not make up answers, just say that you don't know.
                Keep the answer detailed and well formatted based on the information from the context.

                Question:
                {question}

                Context:
                {context}

                Answer:
            """

rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from operator import itemgetter


chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

src_rag_response_chain = (
    {
        "context": (itemgetter('context')
                        |
                    RunnableLambda(format_docs)),
        "question": itemgetter("question")
    }
        |
    rag_prompt_template
        |
    chatgpt
        |
    StrOutputParser()
)

rag_chain_w_sources = (
    {
        "context": similarity_retriever,
        "question": RunnablePassthrough()
    }
        |
    RunnablePassthrough.assign(response=src_rag_response_chain)
)

In [ ]:
query = "What is machine learning?"
result = rag_chain_w_sources.invoke(query)
result

{'context': [Document(metadata={'id': '564928', 'page': 1, 'source': 'Wikipedia', 'title': 'Machine learning'}, page_content='Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). It is a subfield of computer science. The idea came from work in artificial intelligence. Machine learning explores the study and construction of algorithms which can learn and make predictions on data. Such algorithms follow programmed instructions, but can also make predictions or decisions based on data. They build a model from sample inputs. Machine learning is done where designing and programming explicit algorithms cannot be done. Examples include spam filtering, detection of network intruders or malicious insiders working towards a data breach, optical character recognition (OCR), search engines and computer vision.'),
  Document(metadata={'id': '359370', 'page': 1, 'source': 'Wikipedia', 'title': 'Supervised learning'}, page_content='In machin

In [ ]:
from IPython.display import display, Markdown

def display_results(result_obj):
    print('Query:')
    display(Markdown(result_obj['question']))
    print()
    print('Response:')
    display(Markdown(result_obj['response']))
    print('='*50)
    print('Sources:')
    for source in result_obj['context']:
        print('Metadata:', source.metadata)
        print('Content Brief:')
        display(Markdown(source.page_content))
        print()


In [ ]:
query = "What is machine learning?"
result = rag_chain_w_sources.invoke(query)
display_results(result)

Query:


What is machine learning?


Response:


Machine learning is a subfield of computer science that provides computers with the ability to learn without being explicitly programmed. The concept originated from work in artificial intelligence and involves the study and construction of algorithms that can learn from and make predictions based on data. These algorithms follow programmed instructions but can also make decisions or predictions based on the data they process. 

In machine learning, a model is built from sample inputs, and it is particularly useful in scenarios where designing and programming explicit algorithms is not feasible. Common applications of machine learning include spam filtering, detection of network intruders, optical character recognition (OCR), search engines, and computer vision.

There are different types of machine learning, including supervised learning, where the system infers a function from labeled training data. In this case, the results of the training are known beforehand, and the system learns to achieve these results correctly, often using inductive reasoning to generalize from the training data.

Additionally, deep learning is a specific kind of machine learning that primarily utilizes neural networks. It involves multiple layers of processing, allowing the model to learn increasingly abstract representations of the data. Deep learning is particularly effective for complex tasks such as speech recognition, image understanding, and handwriting recognition.

Overall, machine learning enables computers to adapt and improve their performance on tasks through experience, making it a powerful tool in the realm of artificial intelligence.

Sources:
Metadata: {'id': '564928', 'page': 1, 'source': 'Wikipedia', 'title': 'Machine learning'}
Content Brief:


...

Deep learning (also called deep structured learning or hierarchical learning) is a kind of machine learning, which is mostly used with certain kinds of neural networks. As with other kinds of machine-learning, learning sessions can be unsupervised, semi-supervised, or supervised. In many cases, structures are organised so that there is at least one intermediate layer (or hidden layer), between the input layer and the output layer. Certain tasks, such as as recognizing and understanding speech, images or handwriting, is easy to do for humans. However, for a computer, these tasks are very difficult to do. In a multi-layer neural network (having more than two layers), the information processed will become more abstract with each added layer. Deep learning models are inspired by information processing and communication patterns in biological nervous systems; they are different from the structural and functional properties of biological brains (especially the human brain) in many ways, which make them incompatible with neuroscience evidences.


Metadata: {'id': '6360', 'page': 1, 'source': 'Wikipedia', 'title': 'Artificial intelligence'}
Content Brief:


Artificial intelligence (AI) is the ability of a computer program or a machine to think and learn. It is also a field of study which tries to make computers "smart". They work on their own without being encoded with commands. John McCarthy came up with the name "Artificial Intelligence" in 1955. In general use, the term "artificial intelligence" means a programme which mimics human cognition. At least some of the things we associate with other minds, such as learning and problem solving can be done by computers, though not in the same way as we do. Andreas Kaplan and Michael Haenlein define AI as a system’s ability to correctly interpret external data, to learn from such data, and to use those learnings to achieve specific goals and tasks through flexible adaptation. An ideal (perfect) intelligent machine is a flexible agent which perceives its environment and takes actions to maximize its chance of success at some goal or objective. As machines become increasingly capable, mental faculties once thought to require intelligence are removed from the definition. For example, optical character recognition is no longer perceived as an example of "artificial intelligence": it is just a routine technology.


Metadata: {'id': '44742', 'page': 1, 'source': 'Wikipedia', 'title': 'Artificial neural network'}
Content Brief:


A neural network (also called an ANN or an artificial neural network) is a sort of computer software, inspired by biological neurons. Biological brains are capable of solving difficult problems, but each neuron is only responsible for solving a very small part of the problem. Similarly, a neural network is made up of cells that work together to produce a desired result, although each individual cell is only responsible for solving a small part of the problem. This is one method for creating artificially intelligent programs. Neural networks are an example of machine learning, where a program can change as it learns to solve a problem. A neural network can be trained and improved with each example, but the larger the neural network, the more examples it needs to perform well—often needing millions or billions of examples in the case of deep learning. There are two ways to think of a neural network. First is like a human brain. Second is like a mathematical equation.

In [ ]:
query = "What is the difference between AI, ML and DL?"
result = rag_chain_w_sources.invoke(query)
display_results(result)

Query:


What is the difference between AI, ML and DL?


Response:


The difference between AI, ML, and DL can be summarized as follows:

### Artificial Intelligence (AI)
- **Definition**: AI refers to the ability of a computer program or machine to think and learn, mimicking human cognition. It encompasses a broad range of technologies and applications that enable machines to perform tasks that typically require human intelligence.
- **Scope**: AI is a field of study aimed at creating systems that can interpret external data, learn from it, and adapt to achieve specific goals. It includes various subfields, one of which is machine learning.
- **Examples**: AI applications can range from simple rule-based systems to complex algorithms that can learn and make decisions.

### Machine Learning (ML)
- **Definition**: ML is a subfield of AI that focuses on the development of algorithms that allow computers to learn from and make predictions based on data without being explicitly programmed for each task.
- **Functionality**: ML algorithms build models from sample inputs and can improve their performance as they are exposed to more data. They are particularly useful in scenarios where traditional programming is impractical.
- **Examples**: Applications of ML include spam filtering, network intrusion detection, optical character recognition (OCR), and computer vision.

### Deep Learning (DL)
- **Definition**: DL is a specialized subset of machine learning that utilizes multi-layered neural networks to process data. It is often referred to as deep structured learning or hierarchical learning.
- **Structure**: Deep learning models consist of multiple layers (including hidden layers) that allow for the abstraction of information, making them particularly effective for complex tasks such as speech and image recognition.
- **Inspiration**: These models are inspired by the information processing patterns of biological nervous systems, although they differ significantly from the structural and functional properties of human brains.
- **Examples**: DL is commonly used in applications that require high levels of abstraction, such as natural language processing and advanced image recognition.

In summary, AI is the overarching field that includes both ML and DL, with ML being a method of achieving AI through data-driven learning, and DL being a more advanced technique within ML that employs deep neural networks for complex problem-solving.

Sources:
Metadata: {'id': '663523', 'page': 1, 'source': 'Wikipedia', 'title': 'Deep learning'}
Content Brief:


...

Artificial intelligence (AI) is the ability of a computer program or a machine to think and learn. It is also a field of study which tries to make computers "smart". They work on their own without being encoded with commands. John McCarthy came up with the name "Artificial Intelligence" in 1955. In general use, the term "artificial intelligence" means a programme which mimics human cognition. At least some of the things we associate with other minds, such as learning and problem solving can be done by computers, though not in the same way as we do. Andreas Kaplan and Michael Haenlein define AI as a system’s ability to correctly interpret external data, to learn from such data, and to use those learnings to achieve specific goals and tasks through flexible adaptation. An ideal (perfect) intelligent machine is a flexible agent which perceives its environment and takes actions to maximize its chance of success at some goal or objective. As machines become increasingly capable, mental faculties once thought to require intelligence are removed from the definition. For example, optical character recognition is no longer perceived as an example of "artificial intelligence": it is just a routine technology.


Metadata: {'id': '669662', 'page': 1, 'source': 'Wikipedia', 'title': 'Loop AI Labs'}
Content Brief:


Loop AI Labs is an AI and cognitive computing company that focuses on language understanding technology. The company was founded in San Francisco in 2012 by Italian entrepreneur Gianmauro Calafiore, who sold his company Gsmbox to in 2004 and then relocated from Italy to San Francisco. Wanting to start an artificial intelligence company, he recruited two veterans of the project, the largest government-funded AI project in history, who had worked on the project at and Stanford University's . The original company name, "Soshoma", was changed to Loop AI Labs in 2015 after the company decided to change its focus from consumer-oriented to enterprise. Loop AI Labs is headquartered in San Francisco, California, with offices in New York, Milan, and Singapore. The company is privately funded. On May 4, 2017, Loop AI Labs entered into a deal with , a leading European provider of mobile messaging and solutions, to bring their cognitive computing technology to LINK's business clients, which cover 234 million people across Europe.


Metadata: {'id': '44742', 'page': 1, 'source': 'Wikipedia', 'title': 'Artificial neural network'}
Content Brief:


A neural network (also called an ANN or an artificial neural network) is a sort of computer software, inspired by biological neurons. Biological brains are capable of solving difficult problems, but each neuron is only responsible for solving a very small part of the problem. Similarly, a neural network is made up of cells that work together to produce a desired result, although each individual cell is only responsible for solving a small part of the problem. This is one method for creating artificially intelligent programs. Neural networks are an example of machine learning, where a program can change as it learns to solve a problem. A neural network can be trained and improved with each example, but the larger the neural network, the more examples it needs to perform well—often needing millions or billions of examples in the case of deep learning. There are two ways to think of a neural network. First is like a human brain. Second is like a mathematical equation.

In [ ]:
query = "What is the difference between transformers and vision transformers?"
result = rag_chain_w_sources.invoke(query)
display_results(result)

Query:


What is the difference between transformers and vision transformers?


Response:


The difference between transformers and vision transformers (ViTs) primarily lies in their application and the way they process input data.

1. **Architecture and Input Handling**:
   - **Transformers**: Originally designed for natural language processing (NLP), transformers operate on sequences of tokens (words) and utilize self-attention mechanisms to capture relationships between these tokens. The input is typically a 1D sequence of embeddings.
   - **Vision Transformers (ViTs)**: ViTs adapt the transformer architecture for image classification tasks by treating image patches as tokens. An image is divided into fixed-size patches, which are then flattened and linearly embedded into a sequence. This sequence of patch embeddings is fed into a standard transformer encoder, allowing the model to process 2D image data effectively.

2. **Inductive Biases**:
   - **Transformers**: In NLP, transformers benefit from the sequential nature of text, where the order of tokens is crucial, and they leverage the inherent structure of language.
   - **Vision Transformers**: ViTs lack some of the inductive biases present in convolutional neural networks (CNNs), such as translation equivariance and locality. This means that while CNNs are designed to recognize patterns in spatial hierarchies, ViTs rely on large-scale training to learn these patterns from data.

3. **Performance and Training**:
   - **Transformers**: In NLP, transformers have become the standard due to their ability to scale and perform well on large datasets.
   - **Vision Transformers**: ViTs have shown that when pre-trained on large datasets, they can achieve competitive or superior performance compared to state-of-the-art CNNs, particularly in terms of efficiency and scalability. ViTs can outperform CNNs while requiring significantly less computational resources for training.

4. **Attention Mechanism**:
   - **Transformers**: The self-attention mechanism in transformers allows them to consider the entire sequence of tokens, which is effective for capturing long-range dependencies in text.
   - **Vision Transformers**: ViTs utilize self-attention to integrate information across the entire image, enabling them to focus on semantically important regions for classification tasks. This global attention capability is leveraged even in the early layers of the model.

In summary, while both transformers and vision transformers share a foundational architecture, they differ significantly in their input processing, reliance on inductive biases, performance characteristics, and the application of attention mechanisms tailored to their respective domains.

Sources:
Metadata: {'id': '0a4aa65e-ba24-40c0-890f-7143ff77e6f4', 'page': 0, 'source': './rag_docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf'}
Content Brief:


...

Focuses on the architecture and methodology of the Vision Transformer (ViT), detailing how images are processed by splitting them into patches, embedding them, and utilizing a standard Transformer encoder for image classification tasks. It describes the model's design principles, including the use of position embeddings and the integration of a classification token, while referencing foundational work in Transformer architecture.
Published as a conference paper at ICLR 2021
Transformer Encoder
MLP 
Head
Vision Transformer (ViT)
*
Linear Projection of Flattened Patches
* Extra learnable
     [ cl ass]  embedding
1
2
3
4
5
6
7
8
9
0
Patch + Position 
Embedding
Class
Bird
Ball
Car
...
Embedded 
Patches
Multi-Head 
Attention
Norm
MLP
Norm
+
L x
+
Transformer Encoder
Figure 1: Model overview. We split an image into ﬁxed-size patches, linearly embed each of them,
add position embeddings, and feed the resulting sequence of vectors to a standard Transformer
encoder. In order to perform classiﬁcation, we use the standard approach of adding an extra learnable
“classiﬁcation token” to the sequence. The illustration of the Transformer encoder was inspired by
Vaswani et al. (2017).
3
METHOD
In model design we follow the original Transformer (Vaswani et al., 2017) as closely as possible.
An advantage of this intentionally simple setup is that scalable NLP Transformer architectures – and
their efﬁcient implementations – can be used almost out of the box.
3.1
VISION TRANSFORMER (VIT)
An overview of the model is depicted in Figure 1. The standard Transformer receives as input a 1D
sequence of token embeddings. To handle 2D images, we reshape the image x ∈RH×W ×C into a
sequence of ﬂattened 2D patches xp ∈RN×(P 2·C), where (H, W) is the resolution of the original
image, C is the number of channels, (P, P) is the resolution of each image patch, and N = HW/P 2
is the resulting number of patches, which also serves as the effective input sequence length for the
Transformer. The Transformer uses constant latent vector size D through all of its layers, so we
ﬂatten the patches and map to D dimensions with a trainable linear projection (Eq. 1). We refer to
the output of this projection as the patch embeddings.
Similar to BERT’s [class] token, we prepend a learnable embedding to the sequence of embed-
ded patches (z0
0 = xclass), whose state at the output of the Transformer encoder (z0
L) serves as the
image representation y (Eq. 4). Both during pre-training and ﬁne-tuning, a classiﬁcation head is at-
tached to z0
L. The classiﬁcation head is implemented by a MLP with one hidden layer at pre-training
time and by a single linear layer at ﬁne-tuning time.
Position embeddings are added to the patch embeddings to retain positional information. We use
standard learnable 1D position embeddings, since we have not observed signiﬁcant performance
gains from using more advanced 2D-aware position embeddings (Appendix D.4). The resulting
sequence of embedding vectors serves as input to the encoder.
The Transformer encoder (Vaswani et al., 2017) consists of alternating layers of multiheaded self-
attention (MSA, see Appendix A) and MLP blocks (Eq. 2, 3). Layernorm (LN) is applied before
every block, and residual connections after every block (Wang et al., 2019; Baevski & Auli, 2019).
3


Metadata: {'id': 'c23af995-486e-4fc9-8762-5fb229441a61', 'page': 7, 'source': './rag_docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf'}
Content Brief:


Focuses on the behavior of attention mechanisms in the Vision Transformer (ViT), highlighting how attention distances vary across layers and the implications for feature extraction. It compares the attention patterns in ViT to those in hybrid models that incorporate convolutional networks, suggesting that early layers in CNNs may perform a similar role to localized attention in ViT. Additionally, it notes the relevance of attention distance to the model's ability to focus on semantically important regions for classification tasks.
have consistently small attention distances in the low layers. This highly localized attention is
less pronounced in hybrid models that apply a ResNet before the Transformer (Figure 7, right),
suggesting that it may serve a similar function as early convolutional layers in CNNs. Further, the
attention distance increases with network depth. Globally, we ﬁnd that the model attends to image
regions that are semantically relevant for classiﬁcation (Figure 6).
4.6
SELF-SUPERVISION
Transformers show impressive performance on NLP tasks. However, much of their success stems
not only from their excellent scalability but also from large scale self-supervised pre-training (Devlin
8


Metadata: {'id': '9e785ba7-2575-4e74-a3d6-6edefb9053e8', 'page': 1, 'source': './rag_docs/vision_transformer.pdf', 'title': 'vision_transformer.pdf'}
Content Brief:


Focuses on the performance of the Vision Transformer (ViT) in image recognition tasks, highlighting its ability to achieve state-of-the-art results when pre-trained on large datasets like ImageNet-21k and JFT-300M. It contrasts the inductive biases of convolutional neural networks (CNNs) with the advantages of large-scale training for ViT, demonstrating its competitive accuracy across various benchmarks.
Published as a conference paper at ICLR 2021
inherent to CNNs, such as translation equivariance and locality, and therefore do not generalize well
when trained on insufﬁcient amounts of data.
However, the picture changes if the models are trained on larger datasets (14M-300M images). We
ﬁnd that large scale training trumps inductive bias. Our Vision Transformer (ViT) attains excellent
results when pre-trained at sufﬁcient scale and transferred to tasks with fewer datapoints. When
pre-trained on the public ImageNet-21k dataset or the in-house JFT-300M dataset, ViT approaches
or beats state of the art on multiple image recognition benchmarks. In particular, the best model
reaches the accuracy of 88.55% on ImageNet, 90.72% on ImageNet-ReaL, 94.55% on CIFAR-100,
and 77.63% on the VTAB suite of 19 tasks.
2
RELATED WORK
Transformers were proposed by Vaswani et al. (2017) for machine translation, and have since be-
come the state of the art method in many NLP tasks. Large Transformer-based models are often
pre-trained on large corpora and then ﬁne-tuned for the task at hand: BERT (Devlin et al., 2019)
uses a denoising self-supervised pre-training task, while the GPT line of work uses language mod-
eling as its pre-training task (Radford et al., 2018; 2019; Brown et al., 2020).
Naive application of self-attention to images would require that each pixel attends to every other
pixel. With quadratic cost in the number of pixels, this does not scale to realistic input sizes. Thus,
to apply Transformers in the context of image processing, several approximations have been tried in
the past. Parmar et al. (2018) applied the self-attention only in local neighborhoods for each query
pixel instead of globally. Such local multi-head dot-product self attention blocks can completely
replace convolutions (Hu et al., 2019; Ramachandran et al., 2019; Zhao et al., 2020). In a different
line of work, Sparse Transformers (Child et al., 2019) employ scalable approximations to global self-
attention in order to be applicable to images. An alternative way to scale attention is to apply it in
blocks of varying sizes (Weissenborn et al., 2019), in the extreme case only along individual axes (Ho
et al., 2019; Wang et al., 2020a). Many of these specialized attention architectures demonstrate
promising results on computer vision tasks, but require complex engineering to be implemented
efﬁciently on hardware accelerators.
Most related to ours is the model of Cordonnier et al. (2020), which extracts patches of size 2 × 2
from the input image and applies full self-attention on top. This model is very similar to ViT,
but our work goes further to demonstrate that large scale pre-training makes vanilla transformers
competitive with (or even better than) state-of-the-art CNNs. Moreover, Cordonnier et al. (2020)
use a small patch size of 2 × 2 pixels, which makes the model applicable only to small-resolution
images, while we handle medium-resolution images as well.
There has also been a lot of interest in combining convolutional neural networks (CNNs) with forms
of self-attention, e.g. by augmenting feature maps for image classiﬁcation (Bello et al., 2019) or by
further processing the output of a CNN using self-attention, e.g. for object detection (Hu et al., 2018;
Carion et al., 2020), video processing (Wang et al., 2018; Sun et al., 2019), image classiﬁcation (Wu
et al., 2020), unsupervised object discovery (Locatello et al., 2020), or uniﬁed text-vision tasks (Chen

In [ ]:
query = "What is an Agentic AI System?"
result = rag_chain_w_sources.invoke(query)
display_results(result)

Query:


What is an Agentic AI System?


Response:


The context provided does not contain specific information about an "Agentic AI System." Therefore, I don't know what an Agentic AI System is.

Sources:
Metadata: {'id': '6360', 'page': 1, 'source': 'Wikipedia', 'title': 'Artificial intelligence'}
Content Brief:


...

Swarm Intelligence is a field of Computer science. It is a form of Artificial intelligence. Some animals, mostly insects like ants, or bees form large colonies. These colonies are made of many animals that communicate with each other. Each animal is relatively simple, but by working together with other animals it is able to solve complex tasks. Swarm intelligence wants to obtain similar behaviour than that observed with these animals. Instead of the animals, so called "agents" are used.


Metadata: {'id': '692745', 'page': 1, 'source': 'Wikipedia', 'title': 'Shakey the robot'}
Content Brief:


Shakey the Robot was the first general purpose mobile AI robot. The project combined research in robotics, computer vision, and natural language processing. Because of this, it was the first project that melded logical reasoning and physical action. Shakey was developed at the Artificial Intelligence Center of Stanford Research Institute (now called SRI International) by Nils John Nilsson from 1966 to 1972.


Metadata: {'id': '564218', 'page': 1, 'source': 'Wikipedia', 'title': 'Robot lawyer'}
Content Brief:


A robot lawyer is an artificial intelligence (AI) computer program. It is designed to ask the same questions as a real lawyer about certain legal issues. Robot lawyers are being used in many countries around the world including the United States, the United Kingdom, and Holland.

# Build a RAG System with Source Citations Agentic Pipeline

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = """You are an assistant who is an expert in question-answering tasks.
                Answer the following question using only the following pieces of retrieved context.
                If the answer is not in the context, do not make up answers, just say that you don't know.
                Keep the answer detailed and well formatted based on the information from the context.

                Question:
                {question}

                Context:
                {context}

                Answer:
            """

rag_prompt_template = ChatPromptTemplate.from_template(rag_prompt)
rag_prompt_template.pretty_print()

================================ Human Message =================================

You are an assistant who is an expert in question-answering tasks.
                Answer the following question using only the following pieces of retrieved context.
                If the answer is not in the context, do not make up answers, just say that you don't know.
                Keep the answer detailed and well formatted based on the information from the context.

                Question:
                {question}

                Context:
                {context}

                Answer:
            


In [ ]:
citations_prompt = """You are an assistant who is an expert in analyzing answers to questions
                      and finding out referenced citations from context articles.

                      Given the following question, context and generated answer,
                      analyze the generated answer and quote citations from context articles
                      that can be used to justify the generated answer.

                      Question:
                      {question}

                      Context Articles:
                      {context}

                      Answer:
                      {answer}
                  """

cite_prompt_template = ChatPromptTemplate.from_template(citations_prompt)
cite_prompt_template.pretty_print()

================================ Human Message =================================

You are an assistant who is an expert in analyzing answers to questions
                      and finding out referenced citations from context articles.

                      Given the following question, context and generated answer,
                      analyze the generated answer and quote citations from context articles
                      that can be used to justify the generated answer.

                      Question:
                      {question}

                      Context Articles:
                      {context}

                      Answer:
                      {answer}
                  


In [ ]:
from pydantic import BaseModel, Field
from typing import List

class Citation(BaseModel):
    id: str = Field(description="""The string ID of a SPECIFIC context article
                                   which justifies the answer.""")
    source: str = Field(description="""The source of the SPECIFIC context article
                                       which justifies the answer.""")
    title: str = Field(description="""The title of the SPECIFIC context article
                                      which justifies the answer.""")
    page: int = Field(description="""The page number of the SPECIFIC context article
                                     which justifies the answer.""")
    quotes: str = Field(description="""The VERBATIM sentences from the SPECIFIC context article
                                      that are used to generate the answer.
                                      Should be exact sentences from context article without missing words.""")


class QuotedCitations(BaseModel):
    """Quote citations from given context articles
       that can be used to justify the generated answer. Can be multiple articles."""
    citations: List[Citation] = Field(description="""Citations (can be multiple) from the given
                                                     context articles that justify the answer.""")

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableLambda
from operator import itemgetter


chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
structured_chatgpt = chatgpt.with_structured_output(QuotedCitations)


def format_docs_with_metadata(docs: List[Document]) -> str:
    formatted_docs = [
        f"""Context Article ID: {doc.metadata['id']}
            Context Article Source: {doc.metadata['source']}
            Context Article Title: {doc.metadata['title']}
            Context Article Page: {doc.metadata['page']}
            Context Article Details: {doc.page_content}
         """
            for i, doc in enumerate(docs)
    ]
    return "\n\n" + "\n\n".join(formatted_docs)

rag_response_chain = (
    {
        "context": (itemgetter('context')
                        |
                    RunnableLambda(format_docs_with_metadata)),
        "question": itemgetter("question")
    }
        |
    rag_prompt_template
        |
    chatgpt
        |
    StrOutputParser()
)

cite_response_chain = (
    {
        "context": itemgetter('context'),
        "question": itemgetter("question"),
        "answer": itemgetter("answer")
    }
        |
    cite_prompt_template
        |
    structured_chatgpt
)

rag_chain_w_citations = (
    {
        "context": similarity_retriever,
        "question": RunnablePassthrough()
    }
        |
    RunnablePassthrough.assign(answer=rag_response_chain)
        |
    RunnablePassthrough.assign(citations=cite_response_chain)
)

In [ ]:
query = "What is machine learning"
result = rag_chain_w_citations.invoke(query)
result

{'context': [Document(metadata={'id': '564928', 'page': 1, 'source': 'Wikipedia', 'title': 'Machine learning'}, page_content='Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). It is a subfield of computer science. The idea came from work in artificial intelligence. Machine learning explores the study and construction of algorithms which can learn and make predictions on data. Such algorithms follow programmed instructions, but can also make predictions or decisions based on data. They build a model from sample inputs. Machine learning is done where designing and programming explicit algorithms cannot be done. Examples include spam filtering, detection of network intruders or malicious insiders working towards a data breach, optical character recognition (OCR), search engines and computer vision.'),
  Document(metadata={'id': '359370', 'page': 1, 'source': 'Wikipedia', 'title': 'Supervised learning'}, page_content='In machin

In [ ]:
result['citations'].dict()['citations']

[{'id': '564928',
  'source': 'Wikipedia',
  'title': 'Machine learning',
  'page': 1,
  'quotes': 'Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). It is a subfield of computer science. The idea came from work in artificial intelligence. Machine learning explores the study and construction of algorithms which can learn and make predictions on data.'},
 {'id': '564928',
  'source': 'Wikipedia',
  'title': 'Machine learning',
  'page': 1,
  'quotes': 'Such algorithms follow programmed instructions, but can also make predictions or decisions based on data. They build a model from sample inputs.'},
 {'id': '564928',
  'source': 'Wikipedia',
  'title': 'Machine learning',
  'page': 1,
  'quotes': 'Examples include spam filtering, detection of network intruders or malicious insiders working towards a data breach, optical character recognition (OCR), search engines and computer vision.'},
 {'id': '6360',
  'source': 'Wikipedia',

In [ ]:
import re
# used mostly for nice display formatting, ignore if not needed
def get_cited_context(result_obj):
    # Dictionary to hold separate citation information for each unique source and title combination
    source_with_citations = {}

    def highlight_text(context, quote):
        # Normalize whitespace and remove unnecessary punctuation
        quote = re.sub(r'\s+', ' ', quote).strip()
        context = re.sub(r'\s+', ' ', context).strip()

        # Split quote into phrases, being careful with punctuation
        phrases = [phrase.strip() for phrase in re.split(r'[.!?]', quote) if phrase.strip()]

        highlighted_context = context

        for phrase in phrases: # for each quoted phrase

            # Create regex pattern to match cited phrases
            # Escape special regex characters, but preserve word boundaries
            escaped_phrase = re.escape(phrase)
            # Create regex pattern that allows for slight variations
            pattern = re.compile(r'\b' + escaped_phrase + r'\b', re.IGNORECASE)

            # Replace all matched phrases with bolded version
            highlighted_context = pattern.sub(lambda m: f"**{m.group(0)}**", highlighted_context)

        return highlighted_context

    # Process the citation data
    for cite in result_obj['citations'].dict()['citations']:
        cite_id = cite['id']
        title = cite['title']
        source = cite['source']
        page = cite['page']
        quote = cite['quotes']

        # Check if the (source, title) key exists, and initialize if it doesn't
        if (source, title) not in source_with_citations:
            source_with_citations[(source, title)] = {
                'title': title,
                'source': source,
                'citations': []
            }

        # Find or create the citation entry for this unique (id, page) combination
        citation_entry = next(
            (c for c in source_with_citations[(source, title)]['citations'] if c['id'] == cite_id and c['page'] == page),
            None
        )
        if citation_entry is None:
            citation_entry = {'id': cite_id, 'page': page, 'quote': [quote], 'context': None}
            source_with_citations[(source, title)]['citations'].append(citation_entry)
        else:
            citation_entry['quote'].append(quote)

    # Process context data
    for context in result_obj['context']:
        context_id = context.metadata['id']
        context_page = context.metadata['page']
        source = context.metadata['source']
        title = context.metadata['title']
        page_content = context.page_content

        # Match the context to the correct citation entry by source, title, id, and page
        if (source, title) in source_with_citations:
            for citation in source_with_citations[(source, title)]['citations']:
                if citation['id'] == context_id and citation['page'] == context_page:
                    # Apply highlighting for each quote in the citation's quote list
                    highlighted_content = page_content
                    for quote in citation['quote']:
                        highlighted_content = highlight_text(highlighted_content, quote)
                    citation['context'] = highlighted_content

    # Convert the dictionary to a list of dictionaries for separate entries
    final_result_list = [
        {
            'title': details['title'],
            'source': details['source'],
            'citations': details['citations']
        }
        for details in source_with_citations.values()
    ]

    return final_result_list


In [ ]:
get_cited_context(result)

[{'title': 'Machine learning',
  'source': 'Wikipedia',
  'citations': [{'id': '564928',
    'page': 1,
    'quote': ['Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). It is a subfield of computer science. The idea came from work in artificial intelligence. Machine learning explores the study and construction of algorithms which can learn and make predictions on data.',
     'Such algorithms follow programmed instructions, but can also make predictions or decisions based on data. They build a model from sample inputs.',
     'Examples include spam filtering, detection of network intruders or malicious insiders working towards a data breach, optical character recognition (OCR), search engines and computer vision.'],
    'context': 'Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). **It is a subfield of computer science**. **The idea came from work in artificial 

In [ ]:
from IPython.display import display, Markdown

def display_results(result_obj):
    print('Query:')
    display(Markdown(result_obj['question']))
    print()
    print('Response:')
    display(Markdown(result_obj['answer']))
    print('='*50)
    print('Sources:')
    cited_context = get_cited_context(result_obj)
    for source in cited_context:
        print('Title:', source['title'], ' ', 'Source:', source['source'])
        print('Citations:')
        for citation in source['citations']:
            print('ID:', citation['id'], ' ', 'Page:', citation['page'])
            print('Cited Quotes:')
            display(Markdown('*'+' '.join(citation['quote'])+'*'))
            print('Cited Context:')
            display(Markdown(citation['context']))
            print()


In [ ]:
display_results(result)

Query:


What is machine learning


Response:


Machine learning is a subfield of computer science that provides computers with the ability to learn without being explicitly programmed. The concept was introduced by Arthur Samuel in 1959 and is rooted in the broader field of artificial intelligence (AI). Machine learning focuses on the study and construction of algorithms that can learn from and make predictions based on data. 

These algorithms operate by following programmed instructions while also being capable of making predictions or decisions based on the data they process. They build a model from sample inputs, which allows them to perform tasks where traditional programming methods are insufficient. 

Examples of machine learning applications include:
- Spam filtering
- Detection of network intruders or malicious insiders
- Optical character recognition (OCR)
- Search engines
- Computer vision

Overall, machine learning enables systems to improve their performance on tasks over time as they are exposed to more data, making it a powerful tool in various domains.

Sources:
Title: Machine learning   Source: Wikipedia
Citations:
ID: 564928   Page: 1
Cited Quotes:


...

Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). **It is a subfield of computer science**. **The idea came from work in artificial intelligence**. **Machine learning explores the study and construction of algorithms which can learn and make predictions on data**. **Such algorithms follow programmed instructions, but can also make predictions or decisions based on data**. **They build a model from sample inputs**. Machine learning is done where designing and programming explicit algorithms cannot be done. **Examples include spam filtering, detection of network intruders or malicious insiders working towards a data breach, optical character recognition (OCR), search engines and computer vision**.


Title: Artificial intelligence   Source: Wikipedia
Citations:
ID: 6360   Page: 1
Cited Quotes:


*Artificial intelligence (AI) is the ability of a computer program or a machine to think and learn. It is also a field of study which tries to make computers 'smart'.*

Cited Context:


**Artificial intelligence (AI) is the ability of a computer program or a machine to think and learn**. It is also a field of study which tries to make computers "smart". They work on their own without being encoded with commands. John McCarthy came up with the name "Artificial Intelligence" in 1955. In general use, the term "artificial intelligence" means a programme which mimics human cognition. At least some of the things we associate with other minds, such as learning and problem solving can be done by computers, though not in the same way as we do. Andreas Kaplan and Michael Haenlein define AI as a system’s ability to correctly interpret external data, to learn from such data, and to use those learnings to achieve specific goals and tasks through flexible adaptation. An ideal (perfect) intelligent machine is a flexible agent which perceives its environment and takes actions to maximize its chance of success at some goal or objective. As machines become increasingly capable, mental faculties once thought to require intelligence are removed from the definition. For example, optical character recognition is no longer perceived as an example of "artificial intelligence": it is just a routine technology.

In [ ]:
query = "What is AI, ML and DL?"
result = rag_chain_w_citations.invoke(query)
display_results(result)

Query:


What is AI, ML and DL?


Response:


**Artificial Intelligence (AI)**: AI is defined as the ability of a computer program or machine to think and learn. It is also a field of study aimed at making computers "smart," allowing them to operate independently without being explicitly programmed with commands. The term was coined by John McCarthy in 1955. AI encompasses systems that can interpret external data, learn from it, and adapt to achieve specific goals. As technology advances, tasks once considered to require intelligence, such as optical character recognition, are no longer classified as AI but rather as routine technologies.

**Machine Learning (ML)**: ML is a subfield of computer science that provides computers the ability to learn without being explicitly programmed. This concept emerged from the broader field of artificial intelligence. Machine learning focuses on the development of algorithms that can learn from and make predictions based on data. These algorithms can follow programmed instructions while also making decisions based on the data they process. ML is particularly useful in scenarios where traditional programming is impractical, such as spam filtering, network intrusion detection, and computer vision.

**Deep Learning (DL)**: DL is a specialized subset of machine learning that primarily utilizes neural networks, particularly those with multiple layers (known as multi-layer neural networks). It involves learning sessions that can be unsupervised, semi-supervised, or supervised. Deep learning excels in tasks that are challenging for computers, such as speech recognition, image understanding, and handwriting recognition. The architecture of deep learning models is inspired by the information processing patterns of biological nervous systems, although they differ significantly from the structural and functional properties of human brains.

Sources:
Title: Artificial intelligence   Source: Wikipedia
Citations:
ID: 6360   Page: 1
Cited Quotes:


...

Machine learning gives computers the ability to learn without being explicitly programmed (Arthur Samuel, 1959). **It is a subfield of computer science**. **The idea came from work in artificial intelligence**. **Machine learning explores the study and construction of algorithms which can learn and make predictions on data**. **Such algorithms follow programmed instructions, but can also make predictions or decisions based on data**. They build a model from sample inputs. Machine learning is done where designing and programming explicit algorithms cannot be done. Examples include spam filtering, detection of network intruders or malicious insiders working towards a data breach, optical character recognition (OCR), search engines and computer vision.


Title: Deep learning   Source: Wikipedia
Citations:
ID: 663523   Page: 1
Cited Quotes:


*Deep learning (also called deep structured learning or hierarchical learning) is a kind of machine learning, which is mostly used with certain kinds of neural networks. As with other kinds of machine-learning, learning sessions can be unsupervised, semi-supervised, or supervised. In many cases, structures are organised so that there is at least one intermediate layer (or hidden layer), between the input layer and the output layer.*

Cited Context:


**Deep learning (also called deep structured learning or hierarchical learning) is a kind of machine learning, which is mostly used with certain kinds of neural networks**. **As with other kinds of machine-learning, learning sessions can be unsupervised, semi-supervised, or supervised**. **In many cases, structures are organised so that there is at least one intermediate layer (or hidden layer), between the input layer and the output layer**. Certain tasks, such as as recognizing and understanding speech, images or handwriting, is easy to do for humans. However, for a computer, these tasks are very difficult to do. In a multi-layer neural network (having more than two layers), the information processed will become more abstract with each added layer. Deep learning models are inspired by information processing and communication patterns in biological nervous systems; they are different from the structural and functional properties of biological brains (especially the human brain) in many ways, which make them incompatible with neuroscience evidences.

In [ ]:
query = "How is Machine learning related to supervised learning and clustering?"
result = rag_chain_w_citations.invoke(query)
display_results(result)

Query:


How is Machine learning related to supervised learning and clustering?


Response:


Machine learning is a broad field that encompasses various techniques and methodologies for enabling computers to learn from data. Within this field, supervised learning and clustering are two important approaches.

### Supervised Learning
- **Definition**: Supervised learning is a specific type of machine learning where the model is trained on labeled data. This means that the training dataset includes both the input data and the corresponding correct outputs (labels).
- **Process**: The system infers a function from this labeled training data, learning how to map inputs to the correct outputs. The results of the training are known beforehand, allowing the system to learn to produce a "classifier" that can make predictions on new, unseen data.
- **Inductive Reasoning**: Supervised learning typically employs inductive reasoning to generalize from the training data to make predictions on new data.

### Clustering
- **Definition**: Clustering, or cluster analysis, is a type of data analysis that involves grouping a set of objects in such a way that objects in the same group (or cluster) are more similar to each other than to those in other groups.
- **Relation to Machine Learning**: Clustering is often used in data mining and is considered an unsupervised learning technique because it does not rely on labeled data. Instead, it identifies patterns and structures in the data based solely on the inherent characteristics of the data points.

### Relationship Between Machine Learning, Supervised Learning, and Clustering
- **Machine Learning as an Umbrella**: Machine learning serves as the overarching field that includes various learning paradigms, including supervised learning and clustering.
- **Supervised vs. Unsupervised**: While supervised learning requires labeled data and focuses on predicting outcomes based on that data, clustering operates without labels and focuses on discovering inherent groupings within the data.
- **Applications**: Both supervised learning and clustering have practical applications in various domains, such as spam filtering (supervised) and customer segmentation (clustering).

In summary, machine learning encompasses both supervised learning, which relies on labeled data to train models, and clustering, which seeks to find natural groupings in data without predefined labels.

Sources:
Title: Supervised learning   Source: Wikipedia
Citations:
ID: 359370   Page: 1
Cited Quotes:


...

**Clustering or cluster analysis is a type of data analysis**. **The analyst groups objects so that objects in the same group (called a cluster) are more similar to each other than to objects in other groups (clusters) in some way**. This is a common task in data mining.


Title: Reinforcement learning   Source: Wikipedia
Citations:
ID: 610032   Page: 1
Cited Quotes:


*Reinforcement learning is different from supervised learning because the correct inputs and outputs are never shown.*

Cited Context:


Reinforcement learning (RL) is teaching a "software agent" how to behave in an environment by telling it how good it's doing. It is an area of machine learning inspired by behaviorist psychology. **Reinforcement learning is different from supervised learning because the correct inputs and outputs are never shown**. Also, reinforcement learning usually learns as it goes (online learning) unlike supervised learning. This means an agent has to choose between exploring and sticking with what it knows best. A reinforcement learning system is made of a "policy" (formula_1), a "reward function" (formula_2), a "value function" (formula_3), and an optional "model" of the environment.

In [ ]:
query = "What is the difference between transformers and vision transformers?"
result = rag_chain_w_citations.invoke(query)
display_results(result)

Query:


What is the difference between transformers and vision transformers?


Response:


The difference between traditional Transformers and Vision Transformers (ViT) primarily lies in their application and the way they process input data.

### Traditional Transformers
- **Application**: Transformers were originally designed for natural language processing (NLP) tasks. They excel in handling sequential data, where the input is typically a sequence of tokens (words).
- **Input Processing**: In NLP, the input is a 1D sequence of token embeddings. The architecture relies on self-attention mechanisms to capture relationships between tokens, allowing the model to understand context and dependencies in the text.

### Vision Transformers (ViT)
- **Application**: Vision Transformers adapt the Transformer architecture for image classification tasks. They aim to leverage the strengths of Transformers in a domain where convolutional neural networks (CNNs) have traditionally dominated.
- **Input Processing**: Instead of processing a sequence of tokens, ViT processes images by splitting them into fixed-size patches. Each patch is treated as a token, and the sequence of these patch embeddings is fed into a standard Transformer encoder. This approach allows ViT to integrate information across the entire image, similar to how Transformers handle sequences in NLP.
- **Architecture Modifications**: ViT includes additional components such as position embeddings to retain spatial information about the patches, and a classification token that helps in the final classification task.

### Key Advantages of ViT
- **Performance**: When pre-trained on large datasets, ViT can outperform traditional CNNs in terms of efficiency and scalability. It requires significantly less computational resources to achieve comparable or superior performance on image classification tasks.
- **Global Context**: The self-attention mechanism in ViT allows it to capture global context across the entire image, which is a departure from the localized feature extraction typically seen in CNNs.

In summary, while traditional Transformers are designed for sequential data in NLP, Vision Transformers adapt this architecture for image data by treating image patches as tokens, allowing for effective image classification through global attention mechanisms.

Sources:
Title: vision_transformer.pdf   Source: ./rag_docs/vision_transformer.pdf
Citations:
ID: 0a4aa65e-ba24-40c0-890f-7143ff77e6f4   Page: 0
Cited Quotes:


...

Focuses on the behavior of attention mechanisms in the Vision Transformer (ViT), highlighting how attention distances vary across layers and the implications for feature extraction. It compares the attention patterns in ViT to those in hybrid models that incorporate convolutional networks, suggesting that early layers in CNNs may perform a similar role to localized attention in ViT. Additionally, it notes the relevance of attention distance to the model's ability to focus on semantically important regions for classification tasks. have consistently small attention distances in the low layers. This highly localized attention is less pronounced in hybrid models that apply a ResNet before the Transformer (Figure 7, right), suggesting that it may serve a similar function as early convolutional layers in CNNs. Further, the attention distance increases with network depth. Globally, we ﬁnd that the model attends to image regions that are semantically relevant for classiﬁcation (Figure 6). 4.6 SELF-SUPERVISION Transformers show impressive performance on NLP tasks. However, much of their success stems not only from their excellent scalability but also from large scale self-supervised pre-training (Devlin 8


ID: 38169299-0ef0-4af7-a2cb-506caedabf6d   Page: 7
Cited Quotes:


*Vision Transformers dominate ResNets on the performance/compute trade-off. ViT uses approximately 2 −4× less compute to attain the same performance.*

Cited Context:


Focuses on a controlled scaling study of various models, including Vision Transformers and ResNets, evaluating their transfer performance from the JFT-300M dataset. It highlights the performance versus pre-training cost, revealing that Vision Transformers generally outperform ResNets in terms of efficiency and scalability, while also discussing the implications for future model scaling efforts. Published as a conference paper at ICLR 2021 4.4 SCALING STUDY We perform a controlled scaling study of different models by evaluating transfer performance from JFT-300M. In this setting data size does not bottleneck the models’ performances, and we assess performance versus pre-training cost of each model. The model set includes: 7 ResNets, R50x1, R50x2 R101x1, R152x1, R152x2, pre-trained for 7 epochs, plus R152x2 and R200x3 pre-trained for 14 epochs; 6 Vision Transformers, ViT-B/32, B/16, L/32, L/16, pre-trained for 7 epochs, plus L/16 and H/14 pre-trained for 14 epochs; and 5 hybrids, R50+ViT-B/32, B/16, L/32, L/16 pre- trained for 7 epochs, plus R50+ViT-L/16 pre-trained for 14 epochs (for hybrids, the number at the end of the model name stands not for the patch size, but for the total dowsampling ratio in the ResNet backbone). Figure 5 contains the transfer performance versus total pre-training compute (see Appendix D.5 for details on computational costs). Detailed results per model are provided in Table 6 in the Ap- pendix. A few patterns can be observed. First, **Vision Transformers dominate ResNets on the performance/compute trade-off**. **ViT uses approximately 2 −4× less compute to attain the same performance** (average over 5 datasets). Second, hybrids slightly outperform ViT at small compu- tational budgets, but the difference vanishes for larger models. This result is somewhat surprising, since one might expect convolutional local feature processing to assist ViT at any size. Third, Vision Transformers appear not to saturate within the range tried, motivating future scaling efforts. 4.5 INSPECTING VISION TRANSFORMER Input Attention Figure 6: Representative ex- amples of attention from the output token to the input space. See Appendix D.7 for details. To begin to understand how the Vision Transformer processes im- age data, we analyze its internal representations. The ﬁrst layer of the Vision Transformer linearly projects the ﬂattened patches into a lower-dimensional space (Eq. 1). Figure 7 (left) shows the top prin- cipal components of the the learned embedding ﬁlters. The com- ponents resemble plausible basis functions for a low-dimensional representation of the ﬁne structure within each patch. After the projection, a learned position embedding is added to the patch representations. Figure 7 (center) shows that the model learns to encode distance within the image in the similarity of position em- beddings, i.e. closer patches tend to have more similar position em- beddings. Further, the row-column structure appears; patches in the same row/column have similar embeddings. Finally, a sinusoidal structure is sometimes apparent for larger grids (Appendix D). That the position embeddings learn to represent 2D image topology ex- plains why hand-crafted 2D-aware embedding variants do not yield improvements (Appendix D.4). Self-attention allows ViT to integrate information across the entire image even in the lowest layers. We investigate to what degree the network makes use of this capability. Speciﬁcally, we compute the average distance in image space across which information is integrated, based on the attention weights (Figure 7, right). This “attention distance” is analogous to receptive ﬁeld size in CNNs. We ﬁnd that some heads attend to most of the image already in the lowest layers, showing that the ability to integrate information globally is indeed used by the model. Other attention heads